# What is CARLA?

CARLA is an open-source autonomous driving simulator. It was built from scratch to serve as a modular and flexible API to address a range of tasks involved in the problem of autonomous driving.

CARLA simulates a highly realistic environment emulating real world towns, cities and highways and the vehicles and other objects that occupy these driving spaces.

### About Training:
The CARLA simulator is a comprehensive solution for producing synthetic training data for applications in autonomous driving (AD) and also other robotics applications. 

### About Testing:
You can deploy the AD agents you have trained within the simulation to test and evaluate their performance and safety, all within the security of a simulated environment, with no risk to hardware or other road users.


# What is its goal?

One of the main goals of CARLA is to help democratize autonomous driving R&D, serving as a tool that can be easily accessed and customized by users (e.g. learning driving policies, training perception algorithms, etc.).


# What's under the hood of Carla?

CARLA is grounded on Unreal Engine to run the simulation and uses the OpenDRIVE standard (1.4 as today) to define roads and urban settings. Control over the simulation is granted through an API handled in Python and C++ that is constantly growing as the project does.

# How  does it work?

The CARLA simulator consists of a scalable client-server architecture.

<img src="./pictures/carla_modules.webp" alt = "Structure of Carla">

## The server:

The server is responsible for everything related with the simulation itself: sensor rendering, computation of physics, updates on the world-state and its actors and much more. As it aims for realistic results, the best fit would be running the server with a dedicated GPU, especially when dealing with machine learning.

## The Client:

The client side consists of a sum of client modules controlling the logic of actors on scene and setting world conditions. This is achieved by leveraging the CARLA API (in Python or C++), a layer that mediates between server and client that is constantly evolving to provide new functionalities.



## Some of the capabilietes of what Carla can achieve:

<b><li>Traffic manager.</b>A built-in system that takes control of the vehicles besides the one used for learning. It acts as a conductor provided by CARLA to recreate urban-like environments with realistic behaviours.
<br><br>
<b><li>Sensors.</b> Vehicles rely on them to dispense information of their surroundings. In CARLA they are a specific kind of actor attached the vehicle and the data they receive can be retrieved and stored to ease the process. Currently the project supports different types of these, <mark  style="background: yellow;
  color:black">from cameras to radars, lidar and many more.</mark>
<br><br>
<b><li>Recorder.</b>This feature is used to reenact a simulation step by step for every actor in the world. It grants access to any moment in the timeline anywhere in the world, making for a great tracing tool.
<br><br>
<b><li>ROS bridge and Autoware implementation.</b>As a matter of universalization, the CARLA project ties knots and works for the integration of the simulator within other learning environments.
<br><br>
<b><li>Open assets.</b>CARLA facilitates different maps for urban settings with <mark  style="background: yellow;
  color:black">control over weather conditions and a blueprint library with a wide set of actors to be used.</mark> However, these elements can be customized and new can be generated following simple guidelines.
<br><br>
<b><li>Scenario runner.</b>In order to ease the learning process for vehicles, CARLA provides a series of routes describing different situations to iterate on. These also set the basis for the CARLA challenge, open for everybody to test their solutions and make it to the leaderboard.

# How to install it?

https://carla.readthedocs.io/en/latest/start_quickstart/

# How to run CARLA?

## 1. Start the server:
<ol>
    <li>Run the executable file either the CarlaUE4.sh or CarlaUE4.exe according to your OS. </li>
    <li>A window containing a view over the city will pop up. This is the spectator view.<br></li>
    <li>To fly around the city use the mouse and WASD keys, holding down the right mouse button to control the direction.</li>
</ol> 

#### The server simulator is now running and waiting for a client to connect and interact with the world

#### To use the server in no rendering mode: 
./CarlaUE4.sh -RenderOffScreen


## 2. Start the client:

You can combine several scripts in order to generate the scenario that you desire, for instance from the .../PythonPI/examples:<br>
<ul>
    <li> to spawn life into the city  run: generate_traffic.py
    <li> to have dynamic weather run: dynamic_weather.py
    <li> to drive a car manually run: manual_control.py
</ul>

### The main advantage of client-server acrhitecture:

This way, the CARLA server can be run on a networked machine, while the python client can be run from a personal computer. This is particularly useful for <mark  style="background: yellow;
  color:black">differentiating the GPU used for running the CARLA simulator and that used for neural network training,</mark> both of which can be highly demanding on graphics hardware.

# The Python API:
For those of you who want to dive directly and deeply on the API, please click <a href="https://carla.readthedocs.io/en/latest/python_api/">here<a>.

# Firtst Project:

We assume the simulator is already running on the server-side, either in rendered mode or in the no-rendering mode.

### The logical structure:
    1. Create the client object
    2. Get the world object
    3. Loading the map
    4. Select the actors and their blueprints
    5. Get the spawn points
    6. Get the Ego vehicle
    7. Set the sensors, recording
    8. Animate actors

## 1. and 2. Create the client and get the world object

In [1]:
import carla
import random

# Connect to the client 
client = carla.Client('localhost', 2000)
# Retrieve the world object
world = client.get_world()

### The Client Object:

The client is the module the user runs to ask for information or changes in the simulation.<br>
A client runs with an IP and a specific port.<br>
It communicates with the server via terminal. There can be many clients running at the same time. Advanced multiclient managing requires thorough understanding of CARLA and <a href="https://carla.readthedocs.io/en/latest/adv_synchrony_timestep/">synchrony</a>.

It serves <b>to maintain the client's connection to the server</b> and has a number of functions for applying commands and loading or exporting data. We can load an alternative map or reload the current one (resetting to initial state) using
the client object.

The <i>Port</i> can be chosen as any available port and is set to 2000 by default on the server, you can also choose a host different from localhost by using a computer's IP address.

### The Wolrd Object:
The world is an object representing the simulation.<br>
It acts as an abstract layer containing the main methods to spawn actors, change the weather, get the current state of the world, etc.<br>
There is only one world per simulation and it will be destroyed and substituted for a new one when the map is changed.

It provides access to all elements of the simulation, including the map, objects within the map, such as buildings, traffic lights, vehicles and pedestrians.

In [2]:
# Get the map
level = world.get_map()
# Get the weather
weather = world.get_weather()
# Get the blueorint library object that contains all the blueprint of the available veichles(cars, bikes, etc.)
blueprint_library = world.get_blueprint_library()


### Synchronous and asynchronous mode
CARLA has a client-server architecture. The server runs the simulation. The client retrieves information and requests changes in the simulation.

By default, CARLA runs in asynchronous mode.

Essentially, in asynchronous mode the CARLA server runs as fast as it can. Client requests are handled on the fly. In synchronous mode the client, running your Python code, takes the reigns and tells the server when to update.

#### When is it convienent to use A-sync mode?
Asynchronous mode is an appropriate mode to run CARLA if you are experimenting or setting up a simulation, so you can fly around the map with the spectator as you place your actors. 

#### When is it convienent to use Sync mode?
When you want to start producing training data or deploying an agent within the simulation, it is advised that you use the synchronous mode since this will give you more control and predictability.



## 3. Loading the map:
The CARLA server normally loads a default map (normally Town10). If you want to launch CARLA with an alternate map, you have two ways:


#### From Server-side:
You can use the config.py script:<br>
./config.py --map Town05 

#### From Client-side:
You can use the world object to load a map from the client:<br>
client.load_world('Town05')

Please find more information about CARLA maps <a href="https://carla.readthedocs.io/en/latest/core_map/">here</a>.

## Spectator navigation
The spectator is a view into the simulation. By default, the spectator opens in a new window when you run the CARLA server on a computer with a screen attached, unless you specify the -RenderOffScreen command line option.

The spectator is helpful to visualize your simulation. Using the spectator, you can familiarize yourself with the map you've loaded, and see the result of any changes you are making, such as adding vehicles, changing the weather, turning on/off various layers of the map and for debugging purposes.

You can fly the spectator around the world using the mouse to control the pitch and yaw of the spectator view and the QWE-ASD keys to move the spectator:

Q - move upwards (towards the top edge of the window)<br>
E - move downwards (towards the lower edge of the window)<br>
W - move forwards<br>
S - move backwards<br>
A - move left<br>
D - move right<br>

Left click and drag the mouse in the spectator window up and down to control pitch and left and right to control yaw.

<img src ="https://carla.readthedocs.io/en/latest/img/tuto_G_getting_started/flying_spectator.gif" alt="Carla Spectator Gif">

#### The spectator and its properties can be accessed and manipulated through the Python API:

In [3]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

# Set the spectator with an empty transform
spectator.set_transform(carla.Transform())
# This will set the spectator at the origin of the map, with 0 degrees
# pitch, yaw and roll - a good way to orient yourself in the map

## Actors and Blueprints
Actors in CARLA are the elements that perform actions within the simulation, and they can affect other actors. Actors in CARLA includes vehicles and walkers and also sensors, traffic signs, traffic lights and the spectator.

The Blueprints are layouts that allow the user to smoothly incorporate new actors into the simulation. They are already-made models with animations and a series of attributes. Some of these are modifiable and others are not.


### Examples of how to manage the blueprint library
The carla.BlueprintLibrary class contains a list of carla.ActorBlueprint elements. It is the world object who can provide access to it.

In [4]:
blueprint_library = world.get_blueprint_library()

Blueprints have an ID to identify them and the actors spawned with it. The library can be read to find a certain ID, choose a blueprint at random, or filter results using a wildcard pattern.

In [5]:
# Find a specific blueprint.
collision_sensor_bp = blueprint_library.find('sensor.other.collision')
# Choose a vehicle blueprint at random.
vehicle_bp = random.choice(blueprint_library.filter('vehicle.*.*'))

Besides that, each carla.ActorBlueprint has a series of carla.ActorAttribute that can be get and set.

In [6]:
# Get the number of wheels of the random veichle
vehicle_bp.get_attribute('number_of_wheels').as_int()

4

In [7]:
veichles = blueprint_library.filter('vehicle.*.*')
bikes = [v for v in veichles if v.get_attribute('number_of_wheels').as_int() == 2 ]

In [8]:
# Use the same color for all the bikes
for b in bikes:
    b.set_attribute('color', '255,0,0')

## Adding Non-player Characters (NCPs):
We now can populate our simulation with some vehicles to simulate a real environment with traffic and other road users or non-player characters (NPCs).

The carla.BlueprintLibrary class contains a list of carla.ActorBlueprint elements. It is the world object who can provide access to it.

### 4. Select the Actors and their Blueprints:
To spawn vehicles, first, we need to select the vehicles we want from the blueprint library. As below:

In [9]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_blueprints = world.get_blueprint_library().filter('*vehicle*')

### 5. Get the Spawn points:
Now we have the blueprints, we need to find some appropriate spots in the map to spawn our vehicles. Each CARLA map provides pre-defined spawn points spread evenly throughout the map on the roads for this purpose.

In [10]:
# Get the map's spawn points
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
for i in range(0,50):
    world.try_spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

### 6. Get the main vehicle or "Ego vehicle":
We need to set a vehicle as the one that will be the centerpoint of our simulation. That is, the vehicle that the autonomous agent will control. In CARLA parlance, we often refer to this vehicle as the "Ego vehicle"

In [11]:
# uncomment for random ego vehicle
ego_vehicle = world.spawn_actor(random.choice(vehicle_blueprints), random.choice(spawn_points))

# uncomment for a specific vehilcle 
#ego_vehicle = world.spawn_actor(bikes[0], random.choice(spawn_points))

In addition to vehicles, CARLA also provides pedestrians to add to simulations to simulate realistic driving scenarios. Vehicles and pedestrians are referred to as actors in the CARLA parlance, learn more about them <a href="https://carla.readthedocs.io/en/latest/core_actors/">here</a>

## 7. Add sensors:
CARLA has models of numerous types of sensors built in to create training data for machine learning. The sensors can be attached to a vehicle, or they can be attached to a fixed point to model for example a CCTV camera.

Here we will attach a standard camera sensor to the ego vehicle to record some video data:

In [12]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

Once we have spawned the camera, we need to set it recording through the <b>listen()</b> method. The listen method takes as argument a callback that defines what to do with the data. You can either stream it to another program or save it to disk.



### Recording on the disk:
We will use a lambda function as a callback to save the data to disk:

In [13]:
# Start camera with PyGame callback
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

This will save the data to the out/ folder <b>as a series of PNG image files</b> named according to the simulation frame number.

There are a multitude of different types of sensors to choose from. <a href="https://carla.readthedocs.io/en/latest/core_sensors/">Here</a> you can delve deeper into the array of sensors available and how to use them

## 8. Animate Actors:
Now we've added our traffic and ego vehicle to the simulation and started recording camera data, we now need to set the vehicles in motion using the <a href="https://carla.readthedocs.io/en/latest/tuto_G_traffic_manager/"><b>Traffic manager</b></a>.<br>

### The Traffic manager: 
It is a component of CARLA that controls vehicles to autonomously move around the roads of the map within the simulation, following the road conventions and behaving like real road users.

We can find all the vehicles in the simulation using the world.get_actors() method, filtering for all the vehicles. We can then use the set_autopilot() method to hand over control of the vehicle to the Traffic Manager

In [14]:
for vehicle in world.get_actors().filter('*vehicle*'):
    vehicle.set_autopilot(True)


Now your simulation is running, with numerous vehicles driving around the map and a camera recording data from one of those vehicles. This data can then be used to feed a machine learning algorithm for training an autonomous driving agent. The Traffic manager has many functions for customising traffic behaviour, learn more here.

Now your simulation is running, with numerous vehicles driving around the map and a camera recording data from one of those vehicles. This data can then be used to feed a machine learning algorithm for training an autonomous driving agent. The Traffic manager has many functions for customising traffic behaviour, learn more here.

#### Stop recording on the camera:

In [15]:
# Record 60 seconds of pictures
import time 
t = time.time()
done = False

while not done:
    if time.time() - t < 60:
        pass
    else:    
        camera.stop()
        done = True
print("Camera has been stopped")

Camera has been stopped
